In [15]:
import numpy as np
from sklearn.decomposition import PCA
import scipy
import scipy.io as sio
import os
import pandas as pd
import matplotlib.pyplot as plt
from collections import namedtuple
from os.path import dirname, join as pjoin

In [36]:
HRTFdata = namedtuple('HRTF',['sid','HRIR_L','HRIR_R','HRTF_L','HRTF_R','ITD','DTF_L','DTF_R','DTF2_L','DTF2_R'])
HRTFdata_full=[]

# Define parameters
fs = 44100
Nbases = [1,5,10,20,30,50,75,100,128]   # number of bases to be selected from
N = 256; # FFT length

# load data
names=os.listdir('C:/Users/root/Documents/00phd/00ThirdPartyCode/ForSignalProcessing/SOFA API/SOFA API for Matlab and Octave 1.1.1/HRTFs/CIPIC_hrtf_database/standard_hrir_database/')
# print(names)
# column_names=['id','elevation','azimuth','hrtf']
# input=pd.DataFrame(columns=column_names)
# input.id=names[1:]
# print(input)

for i in range(1, len(names)-1):  # You can replace 3 with the number of subjects you have
    data_dir = pjoin('C:/Users/root/Documents/00phd/00ThirdPartyCode/ForSignalProcessing/SOFA API/SOFA API for Matlab and Octave 1.1.1/HRTFs/CIPIC_hrtf_database/standard_hrir_database/', names[i])
    mat_fname = pjoin(data_dir, 'hrir_final.mat')
    data=sio.loadmat(mat_fname)
    sid = names[i]
    HRIR_L=data.get('hrir_l')
    HRIR_R=data.get('hrir_r')
    HRTF_L=scipy.fft.rfft(HRIR_L,N)
    HRTF_R=scipy.fft.rfft(HRIR_R,N)
    ITD=data.get('ITD')
    DTF_L=np.zeros(HRTF_L.shape)
    DTF_R=np.zeros(HRTF_L.shape)
    DTF2_L=np.zeros(HRTF_L.shape)
    DTF2_R=np.zeros(HRTF_L.shape)

    # Append data as to the list
    HRTFdata_full.append(HRTFdata(sid,HRIR_L,HRIR_R,HRTF_L,HRTF_R,ITD,DTF_L,DTF_R,DTF2_L,DTF2_R))

print(HRTFdata_full[4].DTF_L.shape)

(25, 50, 129)


In [38]:
print(len(HRTFdata_full))

44


In [42]:
meanHRTF_L=np.zeros((len(HRTFdata_full),HRTFdata_full[0].HRTF_L.shape[2]))
meanHRTF_R=np.zeros((len(HRTFdata_full),HRTFdata_full[0].HRTF_L.shape[2]))

# Compute DTF and DTF2
for i in range(0, len(HRTFdata_full)): # for each subject
    for j in range(0, HRTFdata_full[0].DTF_L.shape[2]): # for each fft bin
        sum_L = np.sum(HRTFdata_full[i].HRTF_L.take(indices=j,axis=2), axis=(0,1)) # unweighted by IID
        sum_R = np.sum(HRTFdata_full[i].HRTF_R.take(indices=j,axis=2), axis=(0,1))

        meanHRTF_L[i][j] = sum_L / (HRTFdata_full[0].HRTF_L.shape[0]*HRTFdata_full[0].HRTF_L.shape[1])
        meanHRTF_R[i][j] = sum_R / (HRTFdata_full[0].HRTF_L.shape[0]*HRTFdata_full[0].HRTF_L.shape[1])
        
        HRTFdata_full[i].DTF_L[:,:,j] = np.divide(HRTFdata_full[i].HRTF_L[:,:,j], meanHRTF_L[i][j])    # element-wise division
        HRTFdata_full[i].DTF_R[:,:,j] = np.divide(HRTFdata_full[i].HRTF_R[:,:,j], meanHRTF_R[i][j])
        
#    subj.meanHRTF(i) = (subj.meanHRTF_l(i)+subj.meanHRTF_r(i)) / 2; # in case u need it


C:\Users\root\AppData\Local\Temp\ipykernel_19936\2862158546.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  meanHRTF_L[i][j] = sum_L / (HRTFdata_full[0].HRTF_L.shape[0]*HRTFdata_full[0].HRTF_L.shape[1])
C:\Users\root\AppData\Local\Temp\ipykernel_19936\2862158546.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  meanHRTF_R[i][j] = sum_R / (HRTFdata_full[0].HRTF_L.shape[0]*HRTFdata_full[0].HRTF_L.shape[1])
C:\Users\root\AppData\Local\Temp\ipykernel_19936\2862158546.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  HRTFdata_full[i].DTF_L[:,:,j] = np.divide(HRTFdata_full[i].HRTF_L[:,:,j], meanHRTF_L[i][j])    # element-wise division
C:\Users\root\AppData\Local\Temp\ipykernel_19936\2862158546.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  HRTFdata_full[i].DTF_R[:,:,j] = np.divide(HRTFdata_full[i].HRTF_R[:,:,j], meanHRTF_R[i][j])


In [43]:
print(len(HRTFdata_full))

44


In [ ]:
# Plot
